
# 📈 Dự đoán giá cổ phiếu (Stock Price Prediction) using Random Forest

> **Lưu ý:** Nên tạo bản sao trước khi chạy để tránh mất dữ liệu gốc.  
> **Note:** Make a copy before running to avoid losing original data.

Trong notebook này, ta sẽ xây dựng mô hình học máy để dự đoán giá cổ phiếu (ví dụ AAPL - Apple).  
In this notebook, we will build a machine learning model to predict stock prices (e.g., AAPL - Apple).


## 📚 Thư viện | Libraries

➡️ Đây là nơi import các thư viện cần thiết để chương trình hoạt động.

➡️ This is where necessary libraries are imported for the program to work.


In [12]:
# 📈 Tải dữ liệu tài chính từ Yahoo Finance
# 📈 Load financial data from Yahoo Finance
import yfinance as yf

# 📊 Xử lý dữ liệu dạng bảng
# 📊 Handle tabular data
import pandas as pd

# 🔢 Tính toán số học với mảng
# 🔢 Perform numerical computations with arrays
import numpy as np

# 📉 Vẽ biểu đồ trực quan hóa dữ liệu
# 📉 Plot charts for data visualization
import matplotlib.pyplot as plt

# 💾 Lưu và tải mô hình ML
# 💾 Save and load ML models
import joblib

# 📚 Chia tập dữ liệu huấn luyện và kiểm tra
# 📚 Split dataset into training and test sets
from sklearn.model_selection import train_test_split

# 🌲 Mô hình hồi quy rừng ngẫu nhiên
# 🌲 Random Forest regression model
from sklearn.ensemble import RandomForestRegressor

# 🧮 Đánh giá độ chính xác bằng sai số tuyệt đối trung bình
# 🧮 Evaluate accuracy using Mean Absolute Error (MAE)
from sklearn.metrics import mean_absolute_error

## 📥 Bước 1: Tải dữ liệu cổ phiếu | Download stock data
➡️ Sử dụng thư viện yfinance để lấy dữ liệu lịch sử giá cổ phiếu của Apple (AAPL) từ Yahoo Finance.

➡️ Use the yfinance library to fetch historical stock price data for Apple (AAPL) from Yahoo Finance.


In [13]:
# 📥 Tải dữ liệu cổ phiếu Apple (AAPL) từ ngày 01/01/2020 đến 01/01/2024
# 📥 Download Apple (AAPL) stock data from Jan 1, 2020 to Jan 1, 2024
import yfinance as yf

data = yf.download("AAPL", start="2020-01-01", end="2024-01-01")

# 👀 In ra 5 dòng đầu tiên để kiểm tra dữ liệu
# 👀 Print the first 5 rows to inspect the data
print(data.head())


[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2020-01-02  72.620857  72.681304  71.373233  71.627107  135480400
2020-01-03  71.914825  72.676454  71.689965  71.847125  146322800
2020-01-06  72.487854  72.526541  70.783256  71.034717  118387200
2020-01-07  72.146927  72.753808  71.926900  72.497514  108872000
2020-01-08  73.307526  73.609760  71.849548  71.849548  132079200



## 🔧 Bước 2: Tính toán các chỉ số kỹ thuật và xử lý dữ liệu  
### Step 2: Calculate technical indicators and process data

📊 Phân tích trung bình động | Moving Average Analysis

    ➡️ Tính trung bình động ngắn hạn (10 ngày) và dài hạn (50 ngày) từ dữ liệu giá đóng cửa cổ phiếu.

    ➡️ Calculate short-term (10-day) and long-term (50-day) moving averages from stock closing price data.

🧠 Chuẩn bị dữ liệu cho mô hình học máy | Prepare data for machine learning

    ➡️ Chọn các đặc trưng kỹ thuật (MA_10, MA_50, Volume) để dự đoán giá đóng cửa của ngày hôm sau.

    ➡️ Select technical features (MA_10, MA_50, Volume) to predict the next day's closing price.

In [14]:
# 📈 Lấy dữ liệu cổ phiếu (Ví dụ: Apple - AAPL)
# 📈 Fetch stock data (Example: Apple - AAPL)
symbol = "AAPL"
data = yf.download(symbol, start="2020-01-01", end="2024-01-01")

# 📊 Tính trung bình động ngắn hạn (10 ngày)
# 📊 Calculate 10-day short-term moving average
data["MA_10"] = data["Close"].rolling(window=10).mean()

# 📊 Tính trung bình động dài hạn (50 ngày)
# 📊 Calculate 50-day long-term moving average
data["MA_50"] = data["Close"].rolling(window=50).mean()

# 👀 Hiển thị 100 dòng đầu tiên của cột Close, MA_10 và MA_50 để kiểm tra
# 👀 Display the first 100 rows of Close, MA_10, and MA_50 columns for inspection
print(data[["Close", "MA_10", "MA_50"]].head(100))

# 🎯 Chọn các đặc trưng đầu vào: MA_10, MA_50 và khối lượng giao dịch
# 🎯 Select input features: MA_10, MA_50, and trading volume
X = data[["MA_10", "MA_50", "Volume"]]

# 🎯 Giá trị mục tiêu: giá đóng cửa ngày hôm sau
# 🎯 Target value: next day's closing price
y = data["Close"].shift(-1)  # Dịch xuống 1 hàng để dự đoán ngày tiếp theo

# 🧹 Loại bỏ dòng cuối vì bị NaN sau khi shift
# 🧹 Remove last row due to NaN from shift
X = X[:-1]
y = y[:-1]

# 🔀 Chia dữ liệu thành tập huấn luyện (80%) và kiểm tra (20%)
# 🔀 Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

[*********************100%***********************]  1 of 1 completed

Price           Close      MA_10      MA_50
Ticker           AAPL                      
Date                                       
2020-01-02  72.620857        NaN        NaN
2020-01-03  71.914825        NaN        NaN
2020-01-06  72.487854        NaN        NaN
2020-01-07  72.146927        NaN        NaN
2020-01-08  73.307526        NaN        NaN
...               ...        ...        ...
2020-05-19  76.098442  75.147946  66.578267
2020-05-20  77.578415  75.619682  66.746729
2020-05-21  77.000023  75.958205  66.951658
2020-05-22  77.495781  76.171089  67.298347
2020-05-26  76.970848  76.212888  67.490381

[100 rows x 3 columns]



## 🤖 Bước 3: Huấn luyện và đánh giá mô hình  
### Step 3: Train and Evaluate the model

    ➡️ Sử dụng mô hình Random Forest để dự đoán giá đóng cửa ngày hôm sau,
    sau đó đánh giá sai số dự đoán bằng MAE (Mean Absolute Error).

    ➡️ Use Random Forest to predict next day's closing price, then
    evaluate the prediction error using MAE (Mean Absolute Error).


In [15]:
# 🌲 Khởi tạo mô hình Random Forest với 100 cây
# 🌲 Initialize Random Forest model with 100 trees
model = RandomForestRegressor(n_estimators=100, random_state=42)

# 🧠 Huấn luyện mô hình với tập train
# 🧠 Train the model on training data
model.fit(X_train, y_train)

# 🔮 Dự đoán giá cổ phiếu với tập test
# 🔮 Predict stock prices using the test set
y_pred = model.predict(X_test)

# 🔄 Chuyển y_test về dạng mảng numpy để tính toán
# 🔄 Convert y_test to numpy array for evaluation
y_test = np.array(y_test)

# 📏 Tính sai số tuyệt đối trung bình (MAE)
# 📏 Compute Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# 📢 In kết quả sai số
# 📢 Print the error result
print(f"Mean Absolute Error: {mae:.2f}")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Absolute Error: 2.76



## 💾 Bước 4: Lưu mô hình  
### Step 4: Save the trained model

Lưu mô hình vào file để sử dụng lại sau.  
Save the model to a file for future use.


In [16]:
import pickle  # 📦 Thư viện dùng để lưu và tải đối tượng Python
              # 📦 Library used to save and load Python objects

# 💾 Lưu mô hình vào file bằng định dạng nhị phân
# 💾 Save the trained model to file in binary format
with open("stock_model.pkl", "wb") as f:
    pickle.dump(model, f)

# ✅ Thông báo khi lưu thành công
# ✅ Notify when model is successfully saved
print("✅ Đã lưu mô hình huấn luyện (Trained model saved)")


✅ Đã lưu mô hình huấn luyện (Trained model saved)


## 🌐 BƯỚC 5A: Demo Giao diện (trên Internet) | Step 5A: UI Demo (on the Internet)

➡️ Trình bày giao diện web/demo hiển thị kết quả mô hình dự đoán giá cổ phiếu.  
➡️ Present the web interface or demo showing the stock price prediction results.

🔹 Có thể sử dụng các công cụ như:  
🔹 You can use tools such as:
- Streamlit / Gradio để tạo giao diện đơn giản
- Streamlit / Gradio for simple UI
- Flask / FastAPI + HTML/CSS nếu muốn build web chi tiết hơn
- Flask / FastAPI + HTML/CSS for more detailed web apps
- Triển khai trên: Render, Vercel, Hugging Face Spaces, hoặc Replit
- Deploy on: Render, Vercel, Hugging Face Spaces, or Replit

✅ Mục tiêu:
- Người dùng nhập mã cổ phiếu, mô hình hiển thị dự đoán giá tiếp theo
- UI hiển thị biểu đồ đường MA_10, MA_50 kèm giá thực tế và dự đoán

✅ Goal:
- User enters a stock symbol, model shows predicted next-day price
- UI displays line chart of MA_10, MA_50 with actual and predicted prices


### 🚀 Bước 5.1: Cài đặt thư viện cần thiết

**Step 5.1: Install required libraries**

➡️ Để chạy ứng dụng giao diện (UI demo) với Streamlit trên môi trường notebook hoặc Google Colab, bạn cần cài đặt các thư viện sau:  
➡️ To run the UI demo with Streamlit in a notebook or Google Colab, you need to install the following libraries:

    streamlit: Thư viện tạo giao diện web đơn giản cho ứng dụng AI/ML
    → Web UI builder for AI/ML apps

    pyngrok: Dùng để tạo đường dẫn truy cập từ internet (tunnel)
    → Used to expose your local app to the internet via secure tunnels

In [17]:
!pip install streamlit
!pip install pyngrok

###🌐 Bước 5.2: Viết giao diện sử dụng Streamlit

**Step 5.2: Create UI with Streamlit**

➡️ Tạo giao diện web đơn giản cho dự đoán giá cổ phiếu dựa trên mô hình Random Forest và dữ liệu lịch sử từ Yahoo Finance.

➡️ Build a simple web interface for stock price prediction based on Random Forest model and historical data from Yahoo Finance.

In [19]:
# Viết APP Streamlit

%%writefile app.py
# 📦 Import các thư viện cần thiết
# 📦 Import required libraries
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import yfinance as yf

# 💾 Load mô hình đã huấn luyện từ file
# 💾 Load trained model from file
with open("stock_model.pkl", "rb") as f:
    model = pickle.load(f)

# 🔄 Hàm tải dữ liệu cổ phiếu và tính trung bình động
# 🔄 Function to load stock data and calculate moving averages
def load_stock_data(ticker):
    stock_data = yf.download(ticker, start="2020-01-01")
    stock_data["MA_10"] = stock_data["Close"].rolling(window=10).mean()
    stock_data["MA_50"] = stock_data["Close"].rolling(window=50).mean()
    return stock_data

# 🖥️ Giao diện chính Streamlit
# 🖥️ Main Streamlit UI
st.title("📈 Dự đoán giá cổ phiếu với Machine Learning")
ticker = st.text_input("Nhập mã cổ phiếu (ví dụ: AAPL, MSFT, TSLA)", "AAPL").upper()

if st.button("Tải dữ liệu"):
    # 📥 Tải dữ liệu theo mã nhập vào
    # 📥 Load data for input ticker
    data = load_stock_data(ticker)
    st.write("📊 Dữ liệu cổ phiếu:", data.tail())

    # 📉 Vẽ biểu đồ giá đóng cửa và 2 đường MA
    # 📉 Plot closing price and 2 moving averages
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data["Close"], label="Close Price", color="blue")
    ax.plot(data["MA_10"], label="10-day MA", linestyle="dashed", color="green")
    ax.plot(data["MA_50"], label="50-day MA", linestyle="dashed", color="red")
    ax.set_title(f"Biểu đồ giá {ticker}")
    ax.legend()
    st.pyplot(fig)

    # 🔮 Dự đoán giá ngày mai dựa trên dữ liệu mới nhất
    # 🔮 Predict next-day price based on latest data
    latest_data = data[["MA_10", "MA_50", "Volume"]].iloc[-1].values.reshape(1, -1)
    predicted_price = model.predict(latest_data)[0]

    st.success(f"📌 Giá dự đoán ngày mai của {ticker}: **{predicted_price:.2f} USD**")

st.caption("🚀 Ứng dụng Machine Learning dự đoán giá cổ phiếu theo Moving Average + Random Forest.")


Overwriting app.py


### 🔑 Bước 5.3: Đăng nhập và lấy authtoken trên ngrok

**Step 5.3: Sign up and get authtoken on ngrok**

Để hiển thị ứng dụng Streamlit trên Internet thông qua `ngrok`, bạn cần:

To expose your Streamlit app to the Internet via `ngrok`, you need to:

---

1. **Tạo tài khoản ngrok (Sign up for an ngrok account):**  
   Truy cập:  
   https://dashboard.ngrok.com/login?state=0MZdko9buyFsE2RW-1tFrA4KegSYNVU7JMBTL5MJA_luBJvdgi7y3HJkDhj2KUskjAWLa-ObG_Z3DNV1e5I_xlhHktGPw2fSzQUKaPEQCB_vp1wg2WkJm-EBhSPLH7rsgqjDp-b8c1tJz6qnzC7zmV7yYzgdyXSBmHRb6RUnDXzheBwPCFyFgGy-

---

2. **Lấy Authtoken của bạn (Get your authtoken):**  
   Truy cập:  
   https://dashboard.ngrok.com/get-started/your-authtoken

Sau đó, bạn sẽ dùng authtoken này để cấu hình ngrok, giúp tạo đường dẫn truy cập từ Internet đến ứng dụng Streamlit chạy trên máy tính cá nhân hoặc môi trường ảo.

---



### Bước 5.4:

**Step 5.4:**

🔐 Cần đăng nhập trên ngrok để lấy authtoken trước khi dùng

🔐 You must login to ngrok and get authtoken before use

🛠 Đặt authtoken cho ngrok (bạn cần thay bằng authtoken của riêng bạn)

🛠 Set ngrok authtoken (replace with your own authtoken)

In [20]:
# Cần đăng nhập trên ngrok để lấy autoken! (Need to login on ngrok to get autoken!)
# Đây là autoken của tôi, bạn cần tự lấy autoken thay vào! (This is my autoken, you have to auto get autoken!)
!ngrok authtoken "2vnX9rYN7CO6Ah7xA73xsu2HF1R_7JeDiWoAKyKFJHPXc2BiX"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### 🖥️ Bước 5.5: Hiển thị giao diện

---

**Step 5.5: Display the Interface**

---

➡️ Sau khi chạy ứng dụng Streamlit và thiết lập ngrok thành công, bạn sẽ nhận được một URL truy cập công khai.  
➡️ After running the Streamlit app and successfully setting up ngrok, you will get a public URL.

---

➡️ Mở URL này trên trình duyệt để xem giao diện dự đoán giá cổ phiếu trực tuyến.  
➡️ Open this URL in your browser to see the stock price prediction interface online.

---

✅ Tại đây, người dùng có thể:  
- Nhập mã cổ phiếu  
- Tải dữ liệu lịch sử  
- Xem biểu đồ giá và Moving Average  
- Xem kết quả dự đoán giá ngày mai  

In [21]:
# 🚀 Chạy ứng dụng Streamlit và mở cổng 8501 qua ngrok để truy cập từ Internet
# 🚀 Run Streamlit app and open port 8501 via ngrok for internet access

from pyngrok import ngrok

# 🌐 Tạo đường hầm (tunnel) cho cổng 8501 (cổng mặc định của Streamlit)
# 🌐 Create a tunnel for port 8501 (default port for Streamlit)
public_url = ngrok.connect(8501, "http")  # Bỏ "port=" để tránh lỗi

# 📢 In URL truy cập Streamlit qua Internet
# 📢 Print public URL to access Streamlit from Internet
print(f"Truy cập Streamlit tại: {public_url}")

# 🏃‍♂️ Chạy file Streamlit app.py (lệnh shell)
# 🏃‍♂️ Run the Streamlit app.py file (shell command)
!streamlit run app.py


Truy cập Streamlit tại: NgrokTunnel: "https://8340-34-147-16-102.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.147.16.102:8501

YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
  Stopping...
^C


## 🖥️ BƯỚC 5B: Demo giao diện trên máy

**Step 5B: Demo the interface locally**

➡️ Chạy ứng dụng Streamlit trực tiếp trên máy tính cá nhân hoặc môi trường phát triển (local).  

➡️ Run the Streamlit app directly on your local machine or development environment.

> ⚠️ **Lưu ý:**  
> Phần này **phải chạy trên máy cá nhân**, dùng VSCode, PyCharm hoặc bất kỳ IDE nào bạn thích.  
> Không chạy trực tiếp trên Google Colab được vì cần môi trường có giao diện đồ họa và truy cập cổng localhost.  
>
> ⚠️ **Note:**  
> This part **must be run locally** on your personal computer, using VSCode, PyCharm, or any IDE you prefer.  
> It **cannot run directly on Google Colab** because it requires GUI environment and access to localhost ports.


### 🔄 Bước 5.1: Chạy lại phần huấn luyện và lưu model trên máy cá nhân

**Step 5.1: Retrain and save the model locally (not on Colab anymore)**

➡️ Để đảm bảo mô hình được lưu và sử dụng đúng trên máy cá nhân, bạn cần chạy lại toàn bộ phần huấn luyện từ đầu trên môi trường local.  

➡️ To ensure the model is properly saved and used on your local machine, you should rerun the full training process locally.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pickle  # Cần import để lưu mô hình với pickle

# 📈 Lấy dữ liệu cổ phiếu (Ví dụ: Apple - AAPL)
# 📈 Get stock data (Example: Apple - AAPL)
symbol = "AAPL"
data = yf.download(symbol, start="2020-01-01", end="2024-01-01")

# 📊 Tính MA ngắn hạn (10 ngày) và dài hạn (50 ngày)
# 📊 Calculate short-term (10-day) and long-term (50-day) Moving Averages
data["MA_10"] = data["Close"].rolling(window=10).mean()
data["MA_50"] = data["Close"].rolling(window=50).mean()

# 🔍 Hiển thị dữ liệu đầu tiên để kiểm tra
# 🔍 Show first 15 rows to inspect data
print(data[["Close", "MA_10", "MA_50"]].head(15))

# 🧹 Xóa các dòng có giá trị NaN do tính toán Moving Average
# 🧹 Drop rows with NaN values caused by Moving Average calculation
data.dropna(inplace=True)

# 🔧 Tạo biến đầu vào (features) và biến mục tiêu (target)
# 🔧 Create input features and target variable
X = data[["MA_10", "MA_50", "Volume"]]
y = data["Close"].shift(-1)  # Dự đoán giá ngày hôm sau / Predict next day's Close price

# ⚠️ Xóa giá trị NaN cuối cùng do shift(-1)
# ⚠️ Remove last row with NaN target caused by shift
X = X[:-1]
y = y[:-1]

# 🔀 Chia dữ liệu thành train và test (80%-20%)
# 🔀 Split data into training and testing sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🌲 Khởi tạo mô hình Random Forest và huấn luyện
# 🌲 Initialize and train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔮 Dự đoán trên tập test
# 🔮 Predict on the test set
y_pred = model.predict(X_test)

y_test = np.array(y_test)

# 📏 Đánh giá mô hình bằng Mean Absolute Error
# 📏 Evaluate model using Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.2f}")

# 🟢 6. Lưu mô hình vào file để sử dụng lại sau này
# 🟢 Step 6: Save the trained model to a file for later use
with open("stock_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Đã lưu mô hình huấn luyện")


### 🔽 Bước 5.2: Tải thư viện để viết giao diện  

**Step 5.2: Install libraries to build the interface**

➡️ Mở Terminal hoặc Command Prompt trên máy của bạn và chạy lệnh sau:  
➡️ Open your Terminal or Command Prompt and run the following commands:


In [ ]:
pip install streamlit

### 🖥️ Bước 5.3: Viết giao diện, lưu file là **app.py**  
### Step 5.3: Write the interface, save as **app.py**

Dưới đây là ví dụ code giao diện sử dụng Streamlit, load model đã huấn luyện và dự đoán giá cổ phiếu:


In [ ]:
# Giao diện Streamlit để dự đoán giá cổ phiếu
# Streamlit interface for stock price prediction

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import yfinance as yf

# Load mô hình đã huấn luyện từ file
# Load the trained model from file
with open("stock_model.pkl", "rb") as f:
    model = pickle.load(f)

# Hàm tải dữ liệu cổ phiếu, tính MA10 và MA50
# Function to fetch stock data and calculate 10-day & 50-day Moving Averages
def load_stock_data(ticker):
    stock_data = yf.download(ticker, start="2020-01-01")
    stock_data["MA_10"] = stock_data["Close"].rolling(window=10).mean()
    stock_data["MA_50"] = stock_data["Close"].rolling(window=50).mean()
    return stock_data

# Tiêu đề ứng dụng trên giao diện
# App title on the interface
st.title("📈 Dự đoán giá cổ phiếu với Machine Learning")

# Input: mã cổ phiếu từ người dùng
# Input: user inputs stock ticker symbol
ticker = st.text_input("Nhập mã cổ phiếu (ví dụ: AAPL, MSFT, TSLA)", "AAPL").upper()

# Khi người dùng nhấn nút "Tải dữ liệu"
# When user clicks "Tải dữ liệu" (Load data)
if st.button("Tải dữ liệu"):
    data = load_stock_data(ticker)
    st.write("📊 Dữ liệu cổ phiếu:", data.tail())  # Hiển thị 5 dòng cuối dữ liệu

    # Vẽ biểu đồ giá Close và các đường MA
    # Plot Close price and MA lines
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data["Close"], label="Close Price", color="blue")
    ax.plot(data["MA_10"], label="10-day MA", linestyle="dashed", color="green")
    ax.plot(data["MA_50"], label="50-day MA", linestyle="dashed", color="red")
    ax.set_title(f"Biểu đồ giá {ticker}")  # Chart title
    ax.legend()
    st.pyplot(fig)  # Hiển thị biểu đồ trên Streamlit

    # Chuẩn bị dữ liệu mới nhất để dự đoán giá ngày mai
    # Prepare latest features to predict next day price
    latest_data = data[["MA_10", "MA_50", "Volume"]].iloc[-1].values.reshape(1, -1)
    predicted_price = model.predict(latest_data)[0]

    # Hiển thị kết quả dự đoán
    # Display predicted price result
    st.success(f"📌 Giá dự đoán ngày mai của {ticker}: **{predicted_price:.2f} USD**")

# Ghi chú nhỏ phía dưới giao diện
# Small caption at the bottom of the interface
st.caption("🚀 Ứng dụng Machine Learning dự đoán giá cổ phiếu theo Moving Average + Random Forest.")


### 🚀 Bước 5.4: Chạy ứng dụng Streamlit  

**Step 5.4: Run the Streamlit app**

➡️ Mở Terminal hoặc Command Prompt tại thư mục chứa file `app.py`

➡️ Open Terminal or Command Prompt in the folder containing `app.py`

In [ ]:
streamlit run app.py

💡 Lệnh này sẽ khởi động ứng dụng và mở giao diện Streamlit trên trình duyệt.

💡 This command will start the app and open the Streamlit interface in your browser.

